In [334]:
"Jeff: Hi, Mom.  I’m back home. Mom: Today is your first day of school.  How 　(1)　 everything at school? 　Jeff: Not bad.  We have a new PE teacher, Miss Chen.  She’s very good at basketball. Mom: Miss Chen?  Is her name Shu-fen Chen? 　Jeff: That’s right.  　(2)　 you know her before? Mom: She and I 　(3)　 classmates fifteen years ago.  She 　(4)　 the star basketball player at our school.  Hey, let’s go to the park, and you can show me your basketball skills. 　Jeff: OK.  Let’s go!\
（　）(１)(Ａ)　is　(Ｂ)　are　(Ｃ)　was　(Ｄ)　were\
（　）(２)(Ａ)　Do　(Ｂ)　Did　(Ｃ)　Are　(Ｄ)　Were\
（　）(３)(Ａ)　is　(Ｂ)　was　(Ｃ)　are　(Ｄ)　were\
（　）(４)(Ａ)　is　(Ｂ)　was　(Ｃ)　are　(Ｄ)　were\
　Jim: Were you at home last weekend? Meg: No.  I was at my grandparents’ new house.  There’s a nice garden in the front of the house, and my grandpa and I 　(1)　 there all Saturday afternoon. 　Jim: How about your sister, May?  Was she not there with you? Meg: No.  She was 　(2)　 the lake behind the house. 　Jim: 　(3)　 Meg: She played with my grandma’s favorite Cody.  Both she and I love dogs very much.  In my eyes, dogs are cute and faithful. 　Jim: 　(4)　  They’re my favorite animals, too.\
（　）(１)(Ａ)　camped　(Ｂ)　hiked　(Ｃ)　rowed a boat　(Ｄ)　planted one or two trees\
（　）(２)(Ａ)　in　(Ｂ)　on　(Ｃ)　by　(Ｄ)　under\
（　）(３)(Ａ)　What did she do there?　(Ｂ)　What happened to her?　(Ｃ)　Who was she with?　(Ｄ)　Who was with her?\
（　）(４)(Ａ)　Not really.　(Ｂ)　You can say that again!　(Ｃ)　That’s new to me.　(Ｄ)　I see.\
(At Lily’s home) 　Sue: You weren’t at home last weekend.  Where were you? Lily: I 　(1)　 at my grandfather’s new house in the mountains. 　Sue: Is there anything special there? Lily: Yes, there is a pond in front of the house.  There 　(2)　 many tadpoles in the pond then. 　Sue: Tadpoles?  Were they babies of the Taipei tree frogs? Lily: That’s right.  Look at the picture.  The Taipei tree frogs are so small and cute. 　Sue: Can they 　(3)　 trees? Lily: Yes, they can.  They’re protected animals, and people can’t take one home. 　Sue: 　(4)　\
（　）(１)(Ａ)　am　(Ｂ)　was　(Ｃ)　are　(Ｄ)　were\
（　）(２)(Ａ)　is　(Ｂ)　was　(Ｃ)　are　(Ｄ)　were\
（　）(３)(Ａ)　climb　(Ｂ)　hike　(Ｃ)　row　(Ｄ)　jump\
（　）(４)(Ａ)　You can say that again!　(Ｂ)　That’s too bad.  Aren’t they yummy?　(Ｃ)　Good for you!　(Ｄ)　How can that happen?\
Eric: Hi, Dad.  I am home. Dad: How 　(1)　 everything at school today? Eric: Wonderful.  We have a new English teacher, Mr. Wang.  He is very good at English, and he’s so tall and handsome.  He was in the USA ten years 　(2)　, and he’s back to Taiwan this year. Dad: Is his name David Wang?  Is he about 185 centimeters tall? Eric: Yes, that’s right.  When 　(3)　 you know him? Dad: In junior high school.  We were very close friends 　(4)　.  He was also a top basketball player at that time.  You can ask him some basketball skills, too. Eric: Great!\
（　）(１)(Ａ)　is　(Ｂ)　are　(Ｃ)　was　(Ｄ)　were\
（　）(２)(Ａ)　before　(Ｂ)　later　(Ｃ)　after　(Ｄ)　ago\
（　）(３)(Ａ)　do　(Ｂ)　did　(Ｃ)　are　(Ｄ)　were\
（　）(４)(Ａ)　later　(Ｂ)　now　(Ｃ)　ago　(Ｄ)　before\
Joe’s Diary\
Sep. 1, 2018\
　　Today was my first day back at school after the long summer vacation.  We have a new English teacher this semester.  She’s Ms. Lu.  I 　(1)　 English before.  I was never good at it.  But today’s English class 　(2)　 great and lots of fun.  We 　(3)　 and learned some new words.  Everyone in my class 　(4)　.  Now I’m looking forward to learning English well from Ms. Lu.\
（　）(１)(Ａ)　liked　(Ｂ)　was like　(Ｃ)　didn’t like　(Ｄ)　was not like\
（　）(２)(Ａ)　is　(Ｂ)　are　(Ｃ)　was　(Ｄ)　were\
（　）(３)(Ａ)　played a game　(Ｂ)　camped at school　(Ｃ)　hiked for thirty minutes　(Ｄ)　jumped rope\
（　）(４)(Ａ)　was sad　(Ｂ)　was hungry　(Ｃ)　enjoyed the class　(Ｄ)　missed our good old days\
Jim　: Too bad!  Summer vacation ended the day before yesterday.  　(1)　 you have a good time during the vacation? Sue: I sure 　(1)　.  My family and I took a trip to Hualien. Jim　: Really?  What 　(2)　 you do there? Sue: We visited the Ocean Park and 　(3)　 the sea lion show. Jim　: How great!  Maybe my parents can’t be that 　(4)　 during winter vacation, and they can take me there. Sue: I hope so, too.\
（　）(１)(Ａ)　Do; do　(Ｂ)　Are; am　(Ｃ)　Did; did　(Ｄ)　Were; was\
（　）(２)(Ａ)　do　(Ｂ)　did　(Ｃ)　are　(Ｄ)　were\
（　）(３)(Ａ)　watched　(Ｂ)　looked at　(Ｃ)　listened to　(Ｄ)　studied\
（　）(４)(Ａ)　full　(Ｂ)　thirsty　(Ｃ)　free　(Ｄ)　busy\
(In the living room) Dad: Mia, today is February 14.  It’s Valentine’s Day.  　(1)　 Mia　: Yes, I did.  Look at the big box of chocolate.  It’s from my boyfriend, Ted. Dad: Is that all? Mia　: Of course not.  　(2)　 at an expensive restaurant. Dad: Did you go dutch? Mia　: It could not 　(3)　 to me.  Ted’s treat. Dad: 　(4)　 to your mom before. Mia　: Poor Mom. Dad: Haha!  But she loves me for sure.\
（　）(１)(Ａ)　Weren’t you with your boyfriend?　(Ｂ)　Who were you with all night?　(Ｃ)　Did you enjoy it?　(Ｄ)　Did you get flowers or chocolate on this special day?\
（　）(２)(Ａ)　We enjoyed a yummy big meal　(Ｂ)　We helped each other　(Ｃ)　He cooked a big dinner　(Ｄ)　We did not explore or stop\
（　）(３)(Ａ)　show　(Ｂ)　happen　(Ｃ)　join　(Ｄ)　lend\
（　）(４)(Ａ)　I’m always nice　(Ｂ)　I was always nice　(Ｃ)　I never showed that　(Ｄ)　I seldom talked\
答案：(１)(Ｃ)；(２)(Ａ)；(３)(Ｂ)；(４)(Ｃ)
解析：(１)由肯定簡答　Yes, I did.　推知是　Did　起首的　Yes-No　問句，文法及語意只能選(Ｃ)，選項(Ｄ)答句通常是　2　擇　1　作答。
(２)　Mia　和　Ted　在一昂貴的餐廳「享用一頓美味的大餐」。
(３)　Mia　以「各付各的不會發生在她身上。」回應她爸爸問的。
(４)　Mia　感慨「媽好可憐。」可推知「以前她爸爸從不會對她媽媽秀出那樣的。」。
認知向度：(1)閱讀理解能力;(2)閱讀理解能力;(3)閱讀理解能力;(4)閱讀理解能力
出處：試題
課名：There Was a Lot of Trash in the Sea

編號：021264		難易度：難

　Jim: I visited you yesterday morning, but no one was at home.  　(1)　 Ken: I was at my grandparents’ house by the beach. 　Jim: 　(2)　 the beach? Ken: I loved it a lot.  My cousins, Lulu and Hank, and I 　(3)　 beach volleyball after lunch.  After that, we swam in the water.  The beach was clean, and the sea water was warm enough for our favorite water activity. 　Jim: Luckily, the beach is still a good place to 　(4)　.

（　）(１)(Ａ)　What did you do then?　(Ｂ)　Where were you?　(Ｃ)　Weren’t you at home?　(Ｄ)　Did you go out?
（　）(２)(Ａ)　Did you enjoy　(Ｂ)　What do you think about　(Ｃ)　How did you enjoy　(Ｄ)　Do you often go to
（　）(３)(Ａ)　played　(Ｂ)　played the　(Ｃ)　did not play　(Ｄ)　did not play the
（　）(４)(Ａ)　take a shower　(Ｂ)　put trash here and there　(Ｃ)　go on holiday　(Ｄ)　go for a meal
答案：(１)(Ｂ)；(２)(Ｃ)；(３)(Ａ)；(４)(Ｃ)
解析：(１)　Jim　詢問　Ken「你在哪裡？」，Ken　才以「在海邊祖父母家。」回應。
(２)由「我很喜愛它。」可知是　Wh-問句，而且時態是過去式，故選「你多喜愛海灘呢？」。
(３)　play(ed) beach volleyball：打沙灘排球。
(４)很幸運地，海灘仍然是「度假」的好地方；(Ｄ)吃一餐。
認知向度：(1)閱讀理解能力;(2)閱讀理解能力;(3)閱讀理解能力;(4)閱讀理解能力
出處：試題
課名：There Was a Lot of Trash in the Sea

編號：021265		難易度：難

Ken: Today is our first day back to school.  I 　(1)　 all our classmates and the nice teachers.  I’m happy to be here again.  Hey, Roy.  You were in Australia for two weeks in August.  　(2)　 Roy: I joined an English camp. Ken: What?!  I can’t believe it.  You 　(3)　 English before.  How come you joined it? Roy: Australia is my favorite country, and we can see kangaroos and koalas there.  Guess what?  I happened to eat a plate of kangaroo meat for the first time in my life. Ken: 　(4)　 Roy: I sure did.  It was really yummy, but I was sorry for the kangaroo.  It was killed and did not live long. Ken: May God bless it!

（　）(１)(Ａ)　miss　(Ｂ)　stop　(Ｃ)　visit　(Ｄ)　explore
（　）(２)(Ａ)　Who did you go with?　(Ｂ)　How did it happen?　(Ｃ)　What did you do there?　(Ｄ)　Which team did you join?
（　）(３)(Ａ)　didn’t like　(Ｂ)　liked　(Ｃ)　wasn’t like　(Ｄ)　was like
（　）(４)(Ａ)　How did you like it?　(Ｂ)　Didn’t you enjoy it?　(Ｃ)　How about another plate to go?　(Ｄ)　What about one more plate for here?
答案：(１)(Ａ)；(２)(Ｃ)；(３)(Ａ)；(４)(Ｂ)
解析：(１)　Ken　第一天回到學校，很「想念」同學和老師。
(２)　Ken　詢問　Roy「兩星期在澳洲做了什麼？」，Roy　才以「參加了英語營隊。」回應。
(３)　Ken　無法相信　Roy　以前「不喜歡」英文，卻參加了英語營隊。
(４)由　Roy　答句　I sure did.（＝Yes, I did.），可知問句是　Yes-No　問句。
認知向度：(1)閱讀理解能力;(2)閱讀理解能力;(3)閱讀理解能力;(4)閱讀理解能力
出處：試題
課名：There Was a Lot of Trash in the Sea

編號：021266		難易度：難

Mike: Hi, Jill.  Nice to see you.  How were your four-day holidays? 　Jill: I 　(1)　 anything special.  During the day, I 　(2)　 my parents get all the snacks ready.  My parents sell them at the night market. Mike: What a good daughter!  How about the business time?  　(3)　 　Jill: Yes.  The business is usually very good during the holidays.  My parents need some help from me.  How about you? Mike: My parents and I were at Hualien for three days.  I could be with them for so many hours.  How great! 　Jill: 　(4)　  Our family are very important to us.

（　）(１)(Ａ)　do　(Ｂ)　don’t do　(Ｃ)　did　(Ｄ)　didn’t do
（　）(２)(Ａ)　helped　(Ｂ)　helped with　(Ｃ)　didn’t help　(Ｄ)　didn’t help with
（　）(３)(Ａ)　Did you have a good time?　(Ｂ)　Did you give them a hand?　(Ｃ)　Didn’t you eat all the snacks?　(Ｄ)　Didn’t you give them a big hand?
（　）(４)(Ａ)　Walls have ears.　(Ｂ)　An eye for an eye.　(Ｃ)　Let every day be a holiday.　(Ｄ)　You can say that again!
答案：(１)(Ｄ)；(２)(Ａ)；(３)(Ｂ)；(４)(Ｄ)
解析：(１)　anything　通常出現在否定句，而情境是詢問過去假日的事，故選(Ｄ)。
(２)承(１)，Jill「幫忙了」她父母把所有的點心準備好，用過去式作答，選項(Ｂ)其後是接「所幫忙的事」。
(３)　Jill　表示父母在假日需要她的幫忙，故選(Ｂ)。選項(Ｄ)是「妳沒給他們鼓掌嗎？」。
(４)　Mike　很高興能長時間與父母在一起，Jill　也認同並以「一點也沒錯！」回應。(Ａ)隔牆有耳(Ｂ)以眼還眼。
認知向度：(1)閱讀理解能力;(2)閱讀理解能力;(3)閱讀理解能力;(4)閱讀理解能力
出處：試題
課名：There Was a Lot of Trash in the Sea

編號：021267		難易度：中

　　Last Saturday was my sixteenth birthday.  　(1)　 a great party at my place.  My good friends, Mia and Liz, were at the party.  My mom was busy all day, and my dad 　(2)　.  They both cooked such a yummy big dinner.  No wonder everyone 　(3)　 a lot and was very full.  At the end of the party, Mia played the piano, and Liz played the guitar.  Everyone wished me a happy birthday.  I 　(4)　 everyone at the party for the happy time with me.

（　）(１)(Ａ)　There’s　(Ｂ)　There was　(Ｃ)　Here comes　(Ｄ)　There goes
（　）(２)(Ａ)　was, too　(Ｂ)　wasn’t　(Ｃ)　did, too　(Ｄ)　didn’t
（　）(３)(Ａ)　did not like it　(Ｂ)　eats　(Ｃ)　enjoyed it　(Ｄ)　drinks
（　）(４)(Ａ)　kiss　(Ｂ)　look at　(Ｃ)　missed　(Ｄ)　thanked
".strip()


'Lend & Borrow Do you like making snacks?  We do, but we don’t make snacks every day.  Often, we use a bag of baking powder for months.  Join us, and we can lend and borrow ingredients to and from others.  We like to use fresh ingredients to make snacks, right?\u2028‧What do we have in our kitchens now?\u2028－Five bags of flour\u2028－Seven bags of sugar\u2028－Ten bags of salt\u2028－Five boxes of chocolate\u2028－Five bags of marshmallows\u2028－Thirty pineapples\u2028－Fifteen apples\u2028－A lot of bananas\u2028－A lot of oranges\u2028‧Want to join us?\u2028We are mothers in Fun City.\u2028Call us at: 0988-570-991\u2028Or visit us at: www.momsnacklovers.funcity.com（\u3000）(１)\u3000“A marshmallow sandwich is an easy snack.  First, put some fruit on a cookie.  Second, roast a marshmallow over fire.  Third, put another cookie on top.”  What is NOT an ingredient for a marshmallow sandwich?\u3000(Ａ)\u3000Cookies.\u3000(Ｂ)\u3000Fire.\u3000(Ｃ)\u3000Fruit.\u3000(Ｄ)\u3000Marshmallows.（\u3000）(２)\u3

In [264]:
from elasticsearch import Elasticsearch

# Password for the 'elastic' user generated by Elasticsearch
ELASTIC_PASSWORD = "qz666"

# Create the client instance
es = Elasticsearch(
    "http://localhost:9200",
    basic_auth=("elastic", ELASTIC_PASSWORD),
    verify_certs=False,
    ssl_show_warn=False
)

# Successful response!
es.info()

ObjectApiResponse({'name': '40fb61a2e53b', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'MN00F2d8TourjrzCJz2PTw', 'version': {'number': '8.5.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'a846182fa16b4ebfcc89aa3c11a11fd5adf3de04', 'build_date': '2022-11-17T18:56:17.538630285Z', 'build_snapshot': False, 'lucene_version': '9.4.1', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [260]:
import json

body = {
    "settings": {
        "number_of_shards": 3,
        "number_of_replicas": 2
    },
    "mappings": {
        "properties": {
            "index": {
                "type": "keyword"
            },
            "content": {
                "type": "text"
        }
    }
    }
}

es.indices.create(index='english', body=body)

/Users/adam/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'english'})

In [261]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers
import json
import re

def check(_id):
    query = {
        "query": {
            "bool": {
                "must": {
                    "term": {
                        "index": _id
                    }
                }
            }
        }
    }
    result = es.search(index='english', body=query)
    return result['hits']['total']['value'] 
    
with open('corpus.json', 'r')as f:
    data = f.read()
    corpus = eval(data)
    f.close()

def load_datas():
    actions = list()
    for idx in corpus.keys():
        flag = 0 # cloze
        for i, parag in enumerate(corpus[idx]):
            parag = parag.replace("'s", " is").replace("'re", " are").replace("'t", " not")
            parag = re.sub(r'[\u0080-\uffef]', '', parag)
            parag = re.sub(r'[^a-zA-Z]', ' ', parag)
            if len(parag) == 0:
                flag = 1 # reading
                n_cloze = i
                continue
            if flag == 0: # cloze
                index = 'h-' + idx + '-c-' + str(i)
            else: # reading
                index = 'h-' + idx + '-r-' + str(i-n_cloze)
            if check(index) > 0:
                continue
            actions.append({
                "_index": "english",
                "_op_type": "index",
                "_source": {
                    "index": index,
                    "content": parag
                }
            })
    return actions


data = load_datas()
helpers.bulk(es, data)

/Users/adam/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
/Users/adam/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.


(35, [])

In [265]:
q1 = {"query": {
        "multi_match": {
            "query":    "Hi",
            "fields":   [ "content" ]
        }
    }
}

q2 = {"query": {
        "multi_match": {
            "query":    "h-1-1-c-1",
            "fields":   [ "index" ]
        }
    }
}

q3 = {"query": {
        "bool": {
            "must": {
                "term": {
                    "index": "h-1-1-c-1"
                }
            }
        }
    }
}

result = es.search(index='english', body=q3, size=100) # size is maximun number of hits
count = len(result['hits']['hits'])
print(count)

1


/Users/adam/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.


In [266]:
result['hits']['hits']

[{'_index': 'english',
  '_id': 'axDsvoQBWLfKW9tYQxyF',
  '_score': 1.7917595,
  '_source': {'index': 'h-1-1-c-1',
   'content': 'Milly  Hi  Annie   Nice to see you again Annie  Hi  Milly   Nice to see you  too      Milly  Not bad   Thank you       you Annie  Im OK   Thanks Milly  Annie  this is my sister  Sue Annie  Hello  Sue      Sue  Nice to meet you  too     How do you do   Who are you   How are you   What are you     Oh  Really  Hows  And    Nice to meet you   Nice to see you   How are you   How do you do '}}]

In [259]:
# es.options(ignore_status=[400,404]).indices.delete(index='english')

ObjectApiResponse({'acknowledged': True})